# Data Pre Processing

In [1]:
# import packages
import os
import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn import preprocessing 
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
import seaborn as sns
from tqdm.notebook import tqdm_notebook
from tqdm import tqdm
from sklearn.model_selection import RandomizedSearchCV
from IPython.display import display

# pandas display options
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 300)
pd.set_option('float_format', '{:,.4f}'.format) # All float will be displayed in 2 d.p. with comma to separate thousands

In [2]:
# datasets paths
path = "../data"
#path = "/content/drive/MyDrive/cz4041/" # path to Google Drive, for colab
macro = os.path.join(path, "macro.csv")
train = os.path.join(path, "train.zip")
test = os.path.join(path,  "test.zip")

# place all datasets paths in a datasets dict
datasets = {}
datasets['macro'] = macro
datasets['train'] = train
datasets['test'] = test

# load dataframes into dfs dict
dfs = {}
for dataset_name, path in datasets.items():
    df = pd.read_csv(path)
    dfs[dataset_name] = df

# assign to own df variables when you want to use them individually
df_macro = dfs['macro']
df_train = dfs['train']
df_test = dfs['test']


## Overview of datasets
**Dataset size & Number of distinct datatypes**

In [3]:
for dataset_name, df in dfs.items():
    print("====== Dataset size - {}: {} ======".format(dataset_name , df.shape))
    print("Number of distinct datatypes: \n{}".format(df.dtypes.map(type).value_counts()))

====== Dataset size - macro: (2484, 100) ======
Number of distinct datatypes: 
<class 'numpy.dtype[float64]'>    94
<class 'numpy.dtype[object_]'>     4
<class 'numpy.dtype[int64]'>       2
dtype: int64
====== Dataset size - train: (30471, 292) ======
Number of distinct datatypes: 
<class 'numpy.dtype[int64]'>      157
<class 'numpy.dtype[float64]'>    119
<class 'numpy.dtype[object_]'>     16
dtype: int64
====== Dataset size - test: (7662, 291) ======
Number of distinct datatypes: 
<class 'numpy.dtype[int64]'>      159
<class 'numpy.dtype[float64]'>    116
<class 'numpy.dtype[object_]'>     16
dtype: int64


**Prepare dataset for datacleaning**

In [4]:
# Copy train price out to facilitate train & test split later
trainPrice = dfs["train"][["id", "price_doc"]].copy()

# Concat train dataset (minus price_doc) and test dataset for data cleaning
trainNoPrice = dfs["train"].drop("price_doc", axis = 1)

mergeData = pd.concat([trainNoPrice, dfs["test"]])
print(mergeData.shape)


(38133, 291)


## Cleaning of Data

**Since we are predicting price, we will want to drop columns that have low correlation value to 'price_doc'**

In [5]:
# get the correlation to 'price_doc' in train dataset
trainCorr = abs(dfs["train"].corr()["price_doc"])

# pick out features that have less than 5% correlation to be dropped
threshold = trainCorr <= 0.05
lowCorrFeats = trainCorr[threshold]
print(lowCorrFeats, ",", len(lowCorrFeats))

build_year                     0.0022
kitch_sq                       0.0287
school_quota                   0.0140
culture_objects_top_25_raion   0.0443
full_all                       0.0253
male_f                         0.0264
female_f                       0.0243
16_29_all                      0.0223
16_29_male                     0.0231
16_29_female                   0.0216
build_count_block              0.0315
build_count_wood               0.0425
build_count_frame              0.0303
build_count_panel              0.0201
build_count_foam               0.0107
build_count_slag               0.0240
build_count_mix                0.0330
build_count_1921-1945          0.0203
build_count_1971-1995          0.0097
build_count_after_1995         0.0259
cemetery_km                    0.0249
ID_railroad_station_walk       0.0218
water_km                       0.0266
mkad_km                        0.0206
big_market_km                  0.0483
prom_part_500                  0.0090
trc_sqm_500 

In [6]:
# drop
mergeData.drop(list(lowCorrFeats.index), axis = 1, inplace = True)
print(mergeData.shape)

(38133, 247)


In [7]:
# Drop columns with 'ID' in name as they do not provide much value

IDfeats = [feat for feat in mergeData.columns if "ID" in feat]
mergeData.drop(IDfeats, axis = 1, inplace = True)
print(mergeData.shape)

(38133, 241)


In [8]:
# Replace data in the following columns
mergeData.state.replace({33:3},inplace=True)
mergeData["material"].replace(to_replace = 3, value = 1, inplace = True)
mergeData["full_sq"].replace(to_replace = 0, value = np.nan, inplace = True)
mergeData["max_floor"].replace(to_replace = 0, value = np.nan, inplace = True)
mergeData["num_room"].replace(to_replace = 0, value = np.nan, inplace = True)

## Feature Engineering

In [9]:
# create 'year' and 'year_month' features from 'timestamp'
mergeData["year"] = mergeData["timestamp"].apply(lambda x: int(x[0:4]))
mergeData["year_month"] = mergeData["timestamp"].apply(lambda x: x[0:7])\

# create 'living_area_ratio', 'non_living_area' and 'non_living_area_ratio' from 'life_sq' and 'full_sq'
mergeData["living_area_ratio"] = mergeData["life_sq"] / mergeData["full_sq"]
mergeData["non_living_area"] = mergeData["full_sq"] - mergeData["life_sq"]
mergeData["non_living_area_ratio"] = mergeData["non_living_area"] / mergeData["full_sq"]

# create 'room_area_avg' from 'life_sq' and 'num_room'
mergeData["room_area_avg"] = mergeData["life_sq"] / mergeData["num_room"]

# create 'relative_floor' from 'floor' and 'max_floor'
mergeData["relative_floor"] = mergeData["floor"] / mergeData["max_floor"]

# create 'sub_area_building_height_avg' from 'sub_area' and 'max_floor'
sub_area_building_avg = mergeData.groupby('sub_area').agg({'max_floor':np.mean}).reset_index().rename(columns={'max_floor':'sub_area_building_height_avg'})
mergeData = pd.merge(mergeData, sub_area_building_avg, on = ['sub_area'], how = 'left')

# create 'sub_area_kremlin_dist_avg' from 'sub_area' and 'kremlin_km'
kremlin_dist = mergeData.groupby('sub_area').agg({'kremlin_km':np.nanmean}).reset_index().rename(columns={'kremlin_km':'sub_area_kremlin_dist_avg'})
mergeData = pd.merge(mergeData, kremlin_dist, on = ['sub_area'], how = 'left')

# create 'sales_year_month' from 'year_month'
sales_year_month = mergeData.groupby('year_month').size().reset_index().rename(columns={0:'sales_year_month'})
mergeData = pd.merge(mergeData, sales_year_month, on = ['year_month'], how = 'left')

print(mergeData.shape)

(38133, 251)


## Handling Missing Values

In [10]:
# find out columns that have missing values
missing_vals = ((mergeData.isna().sum()))
missing_vals.sort_values(ascending = False, inplace = True)
print(missing_vals[missing_vals > 0])
print("\r\nMissing Values Count: " + str(len(missing_vals[missing_vals > 0])))

hospital_beds_raion                      17859
room_area_avg                            14897
state                                    14253
max_floor                                10355
relative_floor                           10355
num_room                                  9586
material                                  9572
preschool_quota                           8284
cafe_sum_1000_max_price_avg               7746
cafe_sum_1000_min_price_avg               7746
cafe_avg_price_1000                       7746
living_area_ratio                         7562
non_living_area_ratio                     7562
non_living_area                           7562
life_sq                                   7559
build_count_brick                         6209
raion_build_count_with_builddate_info     6209
build_count_before_1920                   6209
build_count_monolith                      6209
build_count_1946-1970                     6209
raion_build_count_with_material_info      6209
cafe_sum_1500

In [11]:
# get the missing values in percentage
missing_vals_pct = ((mergeData.isna().sum()) / len(mergeData))
missing_vals_pct.sort_values(ascending = False, inplace = True)
print(missing_vals_pct[missing_vals_pct > 0])
print("\r\nMissing Values Count: " + str(len(missing_vals_pct[missing_vals_pct > 0])))

hospital_beds_raion                     0.4683
room_area_avg                           0.3907
state                                   0.3738
max_floor                               0.2715
relative_floor                          0.2715
num_room                                0.2514
material                                0.2510
preschool_quota                         0.2172
cafe_sum_1000_max_price_avg             0.2031
cafe_sum_1000_min_price_avg             0.2031
cafe_avg_price_1000                     0.2031
living_area_ratio                       0.1983
non_living_area_ratio                   0.1983
non_living_area                         0.1983
life_sq                                 0.1982
build_count_brick                       0.1628
raion_build_count_with_builddate_info   0.1628
build_count_before_1920                 0.1628
build_count_monolith                    0.1628
build_count_1946-1970                   0.1628
raion_build_count_with_material_info    0.1628
cafe_sum_1500

In [12]:
print(missing_vals_pct[(missing_vals_pct > 0) & (missing_vals_pct <= 0.3000)].index)

for feat in missing_vals_pct[(missing_vals_pct > 0) & (missing_vals_pct <= 0.3000)].index:
    try:
        mergeData[feat].fillna(mergeData[feat].median(), inplace = True)
    except:
        mergeData[feat].fillna(mergeData[feat].mode()[0], inplace = True)

Index(['max_floor', 'relative_floor', 'num_room', 'material',
       'preschool_quota', 'cafe_sum_1000_max_price_avg',
       'cafe_sum_1000_min_price_avg', 'cafe_avg_price_1000',
       'living_area_ratio', 'non_living_area_ratio', 'non_living_area',
       'life_sq', 'build_count_brick', 'raion_build_count_with_builddate_info',
       'build_count_before_1920', 'build_count_monolith',
       'build_count_1946-1970', 'raion_build_count_with_material_info',
       'cafe_sum_1500_min_price_avg', 'cafe_sum_1500_max_price_avg',
       'cafe_avg_price_1500', 'cafe_avg_price_2000',
       'cafe_sum_2000_max_price_avg', 'cafe_sum_2000_min_price_avg',
       'prom_part_5000', 'floor', 'metro_min_walk', 'metro_km_walk',
       'railroad_station_walk_min', 'railroad_station_walk_km', 'product_type',
       'green_part_2000', 'full_sq'],
      dtype='object')


In [13]:
# check remaining missing values in percentage
missing_vals_pct = ((mergeData.isna().sum()) / len(mergeData))
missing_vals_pct.sort_values(ascending = False, inplace = True)
print(missing_vals_pct[missing_vals_pct > 0])
print("\r\nMissing Values Count: " + str(len(missing_vals_pct[missing_vals_pct > 0])))

hospital_beds_raion   0.4683
room_area_avg         0.3907
state                 0.3738
dtype: float64

Missing Values Count: 3


In [14]:
# sklearn KNNImputer
imputer = KNNImputer(n_neighbors=3)

missingCol = list(missing_vals_pct[missing_vals_pct > 0].index)

for i in tqdm_notebook(missingCol):
    mergeData[i] = imputer.fit_transform(mergeData[[i]])

  0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
# check for any remaining missing values
missing_vals_pct = ((mergeData.isna().sum()) / len(mergeData))
missing_vals_pct.sort_values(ascending = False, inplace = True)
print(missing_vals_pct[missing_vals_pct > 0])
print("\r\nMissing Values Count: " + str(len(missing_vals_pct[missing_vals_pct > 0])))
print("\r\n" + str(mergeData.shape))

Series([], dtype: float64)

Missing Values Count: 0

(38133, 251)


## Prepare Train and Test datasets

In [16]:
# Using 'trainPrice' dataset from earlier to split 'mergeData' back into train and test datasets
xTrain = mergeData[mergeData["id"].isin(trainPrice["id"])]
xTrain = pd.merge(xTrain, trainPrice, on = ["id"], how = "inner")

xTest = mergeData[~mergeData["id"].isin(trainPrice["id"])]

yTrain = xTrain["price_doc"].apply(lambda j: np.log1p(j))
xTrain.drop(columns = ["id", "timestamp", "price_doc"], axis = 1, inplace = True)

In [17]:
# split into training and cross-validation
x_tr, x_cv, y_tr, y_cv = train_test_split(xTrain, yTrain, test_size=0.15,random_state=42)

In [18]:
# process categorical ('object') and numerical (non 'object') type data using label encoder

categoricals = xTrain.select_dtypes(include = ["object"]).copy()
numericals = xTrain.select_dtypes(exclude = ["object"])

# categoricals = xTrain.dtypes[(xTrain.dtypes == object)].copy()
# numericals = xTrain.dtypes[(xTrain.dtypes != object)].copy()

# categoricals
for cat in categoricals:
  le = preprocessing.LabelEncoder()
  le.fit(x_tr[cat])

  x_cv[cat] = x_cv[cat].map(lambda s: '<unknown>' if s not in le.classes_ else s)
  le.classes_ = np.append(le.classes_, '<unknown>')

  x_tr[cat] = le.transform(x_tr[cat])
  x_cv[cat] = le.transform(x_cv[cat])

# numericals
for num in numericals:
  min = x_tr[num].min()
  max = x_tr[num].max()
  x_tr[num] = (x_tr[num] - min)/(max-min)
  x_cv[num] = (x_cv[num] - min)/(max-min)

In [19]:
print(x_tr.isnull().values.any())
print(x_cv.isnull().values.any())

False
False


## Random Forest

In [20]:
# Feature Selction
sel = SelectFromModel(RandomForestRegressor(n_jobs=-1,max_depth=10))
sel.fit(x_tr, y_tr)

SelectFromModel(estimator=RandomForestRegressor(max_depth=10, n_jobs=-1))

In [21]:
train_filtered = sel.transform(x_tr)
test_filtered = sel.transform(x_cv)

print(train_filtered.shape,test_filtered.shape)

(25900, 29) (4571, 29)


In [22]:
prams={
     'n_estimators' : [50,100,150,200,250,300],
     'max_depth' : [10,11,12,13,14,15]    
}

random_rf=RandomizedSearchCV(RandomForestRegressor(),param_distributions=prams,verbose=10,n_jobs=-1,cv=2)
random_rf.fit(train_filtered,y_tr)

Fitting 2 folds for each of 10 candidates, totalling 20 fits


RandomizedSearchCV(cv=2, estimator=RandomForestRegressor(), n_jobs=-1,
                   param_distributions={'max_depth': [10, 11, 12, 13, 14, 15],
                                        'n_estimators': [50, 100, 150, 200, 250,
                                                         300]},
                   verbose=10)

In [23]:
print(random_rf.best_params_)
print(random_rf.best_score_)

{'n_estimators': 300, 'max_depth': 12}
0.39352116184361713


In [24]:
# Train the Model with the best hyperparameter
r_cfl=RandomForestRegressor(n_estimators=random_rf.best_params_['n_estimators'],max_depth=random_rf.best_params_['max_depth'],random_state=42,n_jobs=-1)
r_cfl.fit(x_tr,y_tr)

ypred = r_cfl.predict(x_tr)
mse = mean_squared_error(y_tr, ypred)
print("MSE for Train : %.4f" % mse)

ypred = r_cfl.predict(x_cv)
mse = mean_squared_error(y_cv, ypred)
print("MSE for Test : %.4f" % mse)

MSE for Train : 0.1226
MSE for Test : 0.2241


## Output File

In [25]:
test_id = xTest['id']
xTest.drop(['id','timestamp'],axis=1,inplace=True)

print(xTest.shape)

(7662, 249)


C:\Users\huang\Anaconda3\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [26]:
# Predict the House prices
 #Preparing test data:
 
from sklearn.preprocessing import Normalizer
from sklearn import preprocessing

num = xTrain.select_dtypes(exclude=['object'])
cat = xTrain.select_dtypes(include=['object']).copy()


for c in cat:
  le = preprocessing.LabelEncoder()
  le.fit(xTrain[c])

  xTest[c] = xTest[c].map(lambda s: '<unknown>' if s not in le.classes_ else s)
  le.classes_ = np.append(le.classes_, '<unknown>')

  xTest[c] = le.transform(xTest[c])

for c in num:
  min = xTrain[c].min()
  max = xTrain[c].max()

  xTest[c] = (xTest[c] - min)/(max-min)

C:\Users\huang\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\huang\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\huang\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [27]:
#predict the price 
# Testing the Model
test_pred = r_cfl.predict(xTest)

In [28]:
xTest['price_doc'] = np.expm1(test_pred)
xTest.head(10)

C:\Users\huang\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,full_sq,life_sq,floor,max_floor,material,num_room,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_brick,build_count_monolith,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1946-1970,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,...,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,cafe_avg_price_2000,cafe_count_2000_na_price,cafe_count_2000_price_500,cafe_count_2000_price_1000,cafe_count_2000_price_1500,cafe_count_2000_price_2500,cafe_count_2000_price_4000,cafe_count_2000_price_high,big_church_count_2000,church_count_2000,mosque_count_2000,leisure_count_2000,sport_count_2000,market_count_2000,green_part_3000,office_count_3000,office_sqm_3000,trc_count_3000,trc_sqm_3000,cafe_count_3000,cafe_count_3000_na_price,cafe_count_3000_price_500,cafe_count_3000_price_1000,cafe_count_3000_price_1500,cafe_count_3000_price_2500,cafe_count_3000_price_4000,cafe_count_3000_price_high,big_church_count_3000,church_count_3000,mosque_count_3000,leisure_count_3000,sport_count_3000,market_count_3000,green_part_5000,prom_part_5000,office_count_5000,office_sqm_5000,trc_count_5000,trc_sqm_5000,cafe_count_5000,cafe_count_5000_na_price,cafe_count_5000_price_500,cafe_count_5000_price_1000,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,year,year_month,living_area_ratio,non_living_area,non_living_area_ratio,room_area_avg,relative_floor,sub_area_building_height_avg,sub_area_kremlin_dist_avg,sales_year_month,price_doc
30471,0.0071,0.0028,0.0260,0.0690,0.0000,0.0000,0.6667,0,38,0.1180,0.7174,0.1598,0.0788,0.7300,1.0000,0.8462,0.7784,0.9286,0.5000,0.2462,0.1667,0.0000,0.4483,0.2500,0,0.1739,0.0284,0,0,0,0,0,0,0,0,0.7549,0.7727,0.7360,0.7500,0.7399,0.7597,0.4517,0.3794,0.4886,0.7300,0.7443,0.7145,0.7784,0.8013,0.7544,0.7581,0.7767,0.7383,0.7505,0.7700,0.7297,1.0000,0.3690,0.9134,1.0000,0.0916,0.5195,0.0205,0.0098,0.0124,0.0124,0.0027,0.0158,0.0432,0.0310,0.0858,0.0147,0.1774,0.1943,0.1943,...,0.1081,0.0000,0.2014,0.0210,0.0000,0.0000,0.0000,0.0000,0.0027,0.3750,0.3333,0.3493,0.0000,0.0000,0.0000,0.0115,0.0000,0.0000,0.0000,0.0143,0.0185,1.0000,0.0000,0.0926,0.0000,0.1951,0.0000,0.0000,0.0455,0.0275,0.0066,0.0084,0.0045,0.0045,0.0157,0.0000,0.0000,0.0000,0.0098,0.0183,0.5000,0.0000,0.0700,0.0000,0.2510,0.1580,0.0013,0.0030,0.0667,0.0652,0.0072,0.0115,0.0077,0.0062,0.0125,0.0000,0.0000,0.0000,0.0066,0.0400,0.5000,0.0000,0.0642,0.0476,1.0000,47,0.0056,0.5839,0.9944,0.0083,0.0060,0.5248,0.3324,0.2309,"5,127,328.5934"
30472,0.0147,0.0040,0.1039,0.1379,0.0000,0.1111,0.0000,1,103,0.1150,0.0059,0.5810,0.0136,0.0052,0.2416,0.0000,0.0051,0.0000,0.0000,0.2462,0.0000,0.0000,0.0000,0.0000,0,0.0435,0.0000,0,0,0,0,0,0,0,0,0.0052,0.0052,0.0051,0.0058,0.0063,0.0054,0.0055,0.0046,0.0060,0.0052,0.0053

In [29]:
# Dumb the test data with predicted price in to the csv file
xTest['id'] = test_id
xTest[['id','price_doc']].to_csv('output_random_forest.csv',index=False)

C:\Users\huang\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
